In [1]:
import pandas as pd
import numpy as np
import re

### Load data frame from csv file

In [2]:
data_df = pd.read_csv('Mexican_100_recipes_0902.csv')
data_df.head(10)

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
0,Chicken Fajita Stuffed Bell Pepper,['Mexican'],"['Produce', 'Produce', 'Meat', 'Spices and Sea...","['avocado', 'bell pepper', 'chicken breast', '..."
1,Corn Avocado Salsa,['Mexican'],"['Produce', 'Oil, Vinegar, Salad Dressing', 'S...","['avocado', 'balsamic vinegar', 'cumin', 'froz..."
2,Cheesy Chicken Enchilada Quinoa Casserole,['Mexican'],"['Produce', 'Spices and Seasonings', 'Canned a...","['avocado', 'black pepper', 'canned black bean..."
3,Homemade Guacamole,['Mexican'],"['Produce', 'Produce;Spices and Seasonings', '...","['avocado', 'cilantro', 'lime juice', 'black p..."
4,Mini Stuffed Mexican Bell Peppers,['Mexican'],"['Produce', 'Canned and Jarred', 'Spices and S...","['bell pepper', 'canned black beans', 'chili p..."
5,Filet Mignon Soft Tacos,['Mexican'],"['Seafood', 'Oil, Vinegar, Salad Dressing', 'R...","['cod fillets', 'extra virgin olive oil', 'gua..."
6,Instant Pot Chicken Taco Soup,['Mexican'],"['Meat', 'Canned and Jarred', 'Produce', 'Cann...","['boneless skinless chicken breast', 'canned b..."
7,Slow Cooker Chicken Taco Soup,['Mexican'],"['Canned and Jarred', 'Canned and Jarred', 'Ca...","['canned black beans', 'canned diced tomatoes'..."
8,Corn-Crusted Fish Tacos With Jalapeno-Lime Sau...,['Mexican'],"['Canned and Jarred', 'Spices and Seasonings',...","['canned black beans', 'ground cayenne pepper'..."
9,"Grilled Salmon With Cherry, Pineapple, Mango S...",['Mexican'],"['Produce', 'Produce', 'Produce', 'Produce;Spi...","['anaheim pepper', 'cherries', 'fresh basil', ..."


### Define functions required to clean dataframe

In [3]:
# Converting strings to lists then remove [] and ()
def ingredients_cleanup (data_df, column):
    row_list_mexican = []
    indexes = data_df.index.values.tolist()
    counter = 0
    for row in data_df[column]:
        row = row.replace("[", '')
        row = row.replace("'", '')
        row = row.replace("]", '')
        row = row.replace('"', '')
        row_list_mexican = row.split( ',')
        data_df.loc[indexes[counter],column] = row_list_mexican
        counter += 1
    
    return data_df

In [4]:
# Removing leading white spaces 
def remove_leading_ws(data_df, column):
    indexes = data_df.index.values.tolist()
    counter = 0
    for row in data_df[column]:
        for i, element in enumerate(row):
            element = element.strip()
            data_df.loc[indexes[counter],column][i] = element
        counter += 1
    return data_df


In [5]:
# Creating a single list from all the ingredents 
def total_ing_list_from_df(data_df):
    composite_ingredients = []
    for row in data_df.ingredients_SP:
        for element in row:
            composite_ingredients.append(element)
    return composite_ingredients

In [6]:
# Creating list of composite list and creat count of ingredients
def sort_and_count_ingredient_list(ingredient_list):
    ingredient_df = pd.DataFrame(ingredient_list)
    ingredient_df = ingredient_df.rename(columns = {0:  'ingredient'})
    ingredient_df['count'] = 1
    ingredient_counts = ingredient_df.groupby('ingredient').agg({'count':  'count'})
    ingredient_counts.sort_values('count', ascending=False, inplace=True)
    return ingredient_counts


In [7]:
# Replacing different verisons of ingredients
def ingredient_replacement (data_df, replacement_dict):
    
    
    axises = data_df.index.values.tolist()
    counter = 0
    for row in data_df.ingredients_SP:
        for i, element in enumerate(row):
            if element in replacement_dict.keys():
                element = replacement_dict[element]
                data_df.loc[axises[counter],'ingredients_SP'][i] = element
        counter += 1
    return data_df

### Create a new data frame with only Mexican recipes

In [8]:
mexican_df = data_df[data_df['cuisine_SP'] == "['Mexican']"].copy()
mexican_df

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
0,Chicken Fajita Stuffed Bell Pepper,['Mexican'],"['Produce', 'Produce', 'Meat', 'Spices and Sea...","['avocado', 'bell pepper', 'chicken breast', '..."
1,Corn Avocado Salsa,['Mexican'],"['Produce', 'Oil, Vinegar, Salad Dressing', 'S...","['avocado', 'balsamic vinegar', 'cumin', 'froz..."
2,Cheesy Chicken Enchilada Quinoa Casserole,['Mexican'],"['Produce', 'Spices and Seasonings', 'Canned a...","['avocado', 'black pepper', 'canned black bean..."
3,Homemade Guacamole,['Mexican'],"['Produce', 'Produce;Spices and Seasonings', '...","['avocado', 'cilantro', 'lime juice', 'black p..."
4,Mini Stuffed Mexican Bell Peppers,['Mexican'],"['Produce', 'Canned and Jarred', 'Spices and S...","['bell pepper', 'canned black beans', 'chili p..."
...,...,...,...,...
95,Tex-Mex Polenta Rounds with Chunky Guacamole,['Mexican'],"['Produce', 'Spices and Seasonings', 'Cheese',...","['avocado', 'chili powder', 'colby jack', 'gre..."
96,Fresh Mex Salsa,['Mexican'],"['Canned and Jarred', 'Spices and Seasonings',...","['canned black beans', 'chili powder', 'cilant..."
97,Salsa,['Mexican'],"['Gourmet', 'Produce', 'Pasta and Rice;Ethnic ...","['caviar', 'cherry tomato', 'salsa', 'salsa', ..."
98,Artichoke croutons with guacamole,['Mexican'],"['Produce', 'Produce', 'Produce;Spices and Sea...","['avocado', 'cherry tomato', 'parsley', 'onion..."


### For each row (recipe) remove the unwanted characters from the ingredients string, split the string at commas to return a list, save the returned list back to the dataframe.  

In [16]:
# Calling ingredient clean up function to create list
mexican_df = ingredients_cleanup(mexican_df, 'ingredients_SP')
mexican_df = ingredients_cleanup(mexican_df, 'aisle_SP')
mexican_df

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
0,Chicken Fajita Stuffed Bell Pepper,['Mexican'],"[Produce, Produce, Meat, Spices and Seasoni...","[avocado, bell pepper, chicken breast, chil..."
1,Corn Avocado Salsa,['Mexican'],"[Produce, Oil, Vinegar, Salad Dressing, Sp...","[avocado, balsamic vinegar, cumin, frozen c..."
2,Cheesy Chicken Enchilada Quinoa Casserole,['Mexican'],"[Produce, Spices and Seasonings, Canned and ...","[avocado, black pepper, canned black beans, ..."
3,Homemade Guacamole,['Mexican'],"[Produce, Produce;Spices and Seasonings, Pro...","[avocado, cilantro, lime juice, black peppe..."
4,Mini Stuffed Mexican Bell Peppers,['Mexican'],"[Produce, Canned and Jarred, Spices and Seas...","[bell pepper, canned black beans, chili powd..."
...,...,...,...,...
95,Tex-Mex Polenta Rounds with Chunky Guacamole,['Mexican'],"[Produce, Spices and Seasonings, Cheese, Pr...","[avocado, chili powder, colby jack, green p..."
96,Fresh Mex Salsa,['Mexican'],"[Canned and Jarred, Spices and Seasonings, P...","[canned black beans, chili powder, cilantro,..."
97,Salsa,['Mexican'],"[Gourmet, Produce, Pasta and Rice;Ethnic Foo...","[caviar, cherry tomato, salsa, salsa, peac..."
98,Artichoke croutons with guacamole,['Mexican'],"[Produce, Produce, Produce;Spices and Season...","[avocado, cherry tomato, parsley, onion, l..."


### Remove leading white spaces from the ingredients_SP column

In [17]:
mexican_df = remove_leading_ws(mexican_df, 'ingredients_SP')

In [18]:
mexican_df.head()

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
0,Chicken Fajita Stuffed Bell Pepper,['Mexican'],"[Produce, Produce, Meat, Spices and Seasoni...","[avocado, bell pepper, chicken breast, chili p..."
1,Corn Avocado Salsa,['Mexican'],"[Produce, Oil, Vinegar, Salad Dressing, Sp...","[avocado, balsamic vinegar, cumin, frozen corn..."
2,Cheesy Chicken Enchilada Quinoa Casserole,['Mexican'],"[Produce, Spices and Seasonings, Canned and ...","[avocado, black pepper, canned black beans, ca..."
3,Homemade Guacamole,['Mexican'],"[Produce, Produce;Spices and Seasonings, Pro...","[avocado, cilantro, lime juice, black pepper, ..."
4,Mini Stuffed Mexican Bell Peppers,['Mexican'],"[Produce, Canned and Jarred, Spices and Seas...","[bell pepper, canned black beans, chili powder..."


### Create a list of all ingredients from the cuisine 

In [19]:
composite_ingredients_mexican = total_ing_list_from_df(mexican_df)

### Create a dataframe from the composite ingredients list, group by ingredient name with count(), and sort descending

In [20]:
mexican_ing_groups = sort_and_count_ingredient_list(composite_ingredients_mexican)
mexican_ing_groups.to_csv('ingredient_list_mexican_0902.csv')

### Create a dictionary to use to replace different names for the same ingredients.  In final product create reg ex logic to make these changes

In [23]:
replacement_dict = {'coarse sea salt':  'salt_and_pepper',
                    'salsa verde':  'salsa',
                    'boneless skinless chicken breast':  'chicken',
                    'salt':  'salt and pepper',
                    'whole chicken':  'chicken',
                    'white onion':  'onion',
                    'solive oil':  'oil',
                    'palm oil':   'oil',
                    'dried arbol chile':  'chili pepper',
                    'cooking oil':  'oil',
                    'cooked chicken':  'chicken',
                    'yellow onion':  'onion',
                    'ripe banana':'banana',
                    'fresh basil':'basil',
                    'black beans':'beans',
                    'canned kidney beans':'beans',
                    'canned pinto beans':'beans',
                    'chili beans':'beans',
                    'dried black beans':'beans',
                    'refried beans':'beans',
                    '95 percent lean ground beef':'beef',
                    'beef cubes':'beef',
                    'beef shoulder roast':'beef',
                    'flank steak':'beef',
                    'ground chuck':'beef',
                    'round tip steak':'beef',
                    'skirt steak':'beef',
                    'steak':'beef',
                    'green pepper':'bell pepper',
                    'orange pepper':'bell pepper',
                    'red pepper':'bell pepper',
                    'roasted red peppers':'bell pepper',
                    'yellow pepper':'bell pepper',
                    'canned black beans':'beans',
                    'french bread':'bread',
                    'beef broth':'broth',
                    'chicken broth':'broth',
                    'chicken stock':'broth',
                    'low sodium chicken broth':'broth',
                    'red cabbage':'cabbage',
                    'canned chipotle chile':'canned pepper',
                    'canned green chiles':'canned pepper',
                    'chipotle chiles':'canned pepper',
                    'peppers':'fresh peppers',
                    'ground cayenne pepper':'cayenne',
                    'cheddar cheese':'cheese',
                    'colby jack':'cheese',
                    'cotija cheese':'cheese',
                    'cream cheese':'cheese',
                    'extra sharp cheddar cheese':'cheese',
                    'feta cheese':'cheese',
                    'goat cheese':'cheese',
                    'grilling cheese':'cheese',
                    'low fat shredded cheddar':'cheese',
                    'mexican cheese':'cheese',
                    'monterey jack cheese':'cheese',
                    'queso fresco':'cheese',
                    'sharp cheddar cheese':'cheese',
                    'shredded cheddar cheese':'cheese',
                    'shredded cheese':'cheese',
                    'shredded mexican cheese blend':'cheese',
                    'soy cheese':'cheese',
                    'boneless skinless chicken breast':'chicken',
                    'boneless skinless chicken thighs':'chicken',
                    'chicken breast':'chicken',
                    'chicken breast halves':'chicken',
                    'chicken meat':'chicken',
                    'rotisserie chicken':'chicken',
                    'shredded chicken':'chicken',
                    'whole chicken':'chicken',
                    'condensed cream of chicken soup':'chicken soup',
                    'ancho chili powder':'chili powder',
                    'chili pepper':'chili powder',
                    'chili powder':'chili powder',
                    'chili seasoning mix':'chili powder',
                    'ground chipotle chile pepper':'chipotle',
                    'dried cilantro':'cilantro',
                    'cod fillets':'fish',
                    'ground coriander':'coriander',
                    'corn kernels':'corn',
                    'creamed corn':'corn',
                    'frozen corn':'corn',
                    'sweet corn on the cob':'corn',
                    'whole kernel corn':'corn',
                    'mexican crema':'crema',
                    'creole seasoning':'creole season',
                    'persian cucumber':'cucumber',
                    'cumin seeds':'cumin',
                    'ground cumin':'cumin',
                    'dried arbol chile':'dried pepper',
                    'green enchilada sauce':'enchilada sauce',
                    'salmon':'fish',
                    'swordfish':'fish',
                    'tilapia':'fish',
                    'anaheim pepper':'fresh pepper',
                    'green chili pepper':'fresh pepper',
                    'jalapeno pepper':'fresh pepper',
                    'pasilla pepper':'fresh pepper',
                    'poblano pepper':'fresh pepper',
                    'serrano pepper':'fresh pepper',
                    'garlic powder':'garlic',
                    'garlic salt':'garlic',
                    'whole garlic cloves':'garlic',
                    'ginger':'ginger',
                    'guacamole':'guacamole',
                    'smoked ham hock':'ham hock',
                    'light vanilla ice cream':'ice cream',
                    'italian seasoning':'italian season',
                    'lemon juice':'lemon',
                    'lemon peel':'lemon',
                    'butter lettuce leaves':'lettuce',
                    'romaine':'lettuce',
                    'lime juice':'lime',
                    'lime peel':'lime',
                    'lime wedge':'lime',
                    'dried mango':'mango',
                    'condensed cream of mushroom soup':'mushroom soup',
                    'fresh mushrooms':'mushrooms',
                    'mustard powder':'mustard powder',
                    'coconut oil':'oil',
                    'cooking oil':'oil',
                    'extra virgin olive oil':'oil',
                    'grape seed oil':'oil',
                    'light olive oil':'oil',
                    'olive oil':'oil',
                    'peanut oil':'oil',
                    'rice bran oil':'oil',
                    'vegetable oil':'oil',
                    'black olives':'olives',
                    'olives':'olives',
                    'onion powder':'onion',
                    'red onion':'onion',
                    'white onion':'onion',
                    'yellow onion':'onion',
                    'spring onions':'onion',
                    'navel orange':'orange',
                    'orange juice':'orange',
                    'smoked paprika':'paprika',
                    'peas and carrots':'pea',
                    'pico de gallo':'pico de gallo',
                    'pistachio nuts':'pistachio',
                    'boston butt':'pork',
                    'lean pork tenderloin':'pork',
                    'pork chops':'pork',
                    'ground pork sausage':'pork sausage',
                    'yukon gold potato':'potato',
                    'cooked quinoa':'quinoa',
                    'red pepper flakes':'red pepper flake',
                    'brown rice':'rice',
                    'cooked brown rice':'rice',
                    'cooked rice':'rice',
                    'picante sauce':'salsa',
                    'salsa verde':'salsa',
                    'kosher salt':'salt and pepper',
                    'black pepper':'salt and pepper',
                    'coarse sea salt':'salt and pepper',
                    'salt':'salt and pepper',
                    'seasoned salt':'salt and pepper',
                    'white pepper':'salt and pepper',
                    'vegetable stock':'broth',
                    'dark brown sugar':'sugar',
                    'golden brown sugar':'sugar',
                    'dried thyme':'thyme',
                    'canned diced tomatoes':'tomato',
                    'canned tomatoes':'tomato',
                    'cherry tomato':'tomato',
                    'diced tomatoes with green chilies':'tomato',
                    'fire roasted tomatoes':'tomato',
                    'italian tomato':'tomato',
                    'plum tomato':'tomato',
                    'tomato juice':'tomato',
                    'tomato paste':'tomato',
                    'canned tomato sauce':'tomato sauce',
                    'burrito size tortilla':'tortilla',
                    'flour tortilla':'tortilla',
                    'scoops tortilla chips':'tortilla',
                    'white corn tortilla':'tortilla',
                    'wrap':'tortilla',
                    'corn tortilla chips':'tortilla',
                    'ground turkey':'turkey',
                    'apple cider vinegar':'vinegar',
                    'balsamic vinegar':'vinegar',
                    'distilled white vinegar':'vinegar',
                    'white wine vinegar':'vinegar',
                    'fat free greek yogurt':'yogurt',
                    'greek yogurt':'yogurt',
                    'low fat yogurt':'yogurt',

}

### Use the replacement dictionary to replace the keys with the values

In [24]:
mexican_df = ingredient_replacement(mexican_df, replacement_dict)
mexican_df

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
0,Chicken Fajita Stuffed Bell Pepper,['Mexican'],"[Produce, Produce, Meat, Spices and Seasoni...","[avocado, bell pepper, chicken, chili powder, ..."
1,Corn Avocado Salsa,['Mexican'],"[Produce, Oil, Vinegar, Salad Dressing, Sp...","[avocado, vinegar, cumin, corn, garlic, bell p..."
2,Cheesy Chicken Enchilada Quinoa Casserole,['Mexican'],"[Produce, Spices and Seasonings, Canned and ...","[avocado, salt and pepper, beans, tomato, chil..."
3,Homemade Guacamole,['Mexican'],"[Produce, Produce;Spices and Seasonings, Pro...","[avocado, cilantro, lime, salt and pepper, oni..."
4,Mini Stuffed Mexican Bell Peppers,['Mexican'],"[Produce, Canned and Jarred, Spices and Seas...","[bell pepper, beans, chili powder, canned pepp..."
...,...,...,...,...
95,Tex-Mex Polenta Rounds with Chunky Guacamole,['Mexican'],"[Produce, Spices and Seasonings, Cheese, Pr...","[avocado, chili powder, cheese, bell pepper, p..."
96,Fresh Mex Salsa,['Mexican'],"[Canned and Jarred, Spices and Seasonings, P...","[beans, chili powder, cilantro, cumin, corn, b..."
97,Salsa,['Mexican'],"[Gourmet, Produce, Pasta and Rice;Ethnic Foo...","[caviar, tomato, salsa, salsa, peach, salsa, s..."
98,Artichoke croutons with guacamole,['Mexican'],"[Produce, Produce, Produce;Spices and Season...","[avocado, tomato, parsley, onion, lemon, salt ..."


### Create a new composite ingredient list from the data frame

In [25]:
composite_ingredients_mexican = total_ing_list_from_df(mexican_df)

### Create a dataframe from the composite ingredients list, group by ingredient name with count(), and sort descending

In [26]:
mexican_ing_groups = sort_and_count_ingredient_list(composite_ingredients_mexican)
mexican_ing_groups.head(20)

,count
ingredient,
salt and pepper,103
onion,98
lime,66
garlic,65
tomato,64
cilantro,54
oil,52
fresh pepper,46
cheese,44


In [59]:
# Dropping common ingredients 
mexican_ing_groups = mexican_ing_groups.drop(index = ["salt and pepper", "oil", "tortilla"])
mexican_ing_groups

,count
ingredient,
onion,98
lime,66
garlic,65
tomato,64
cilantro,54
...,...
port,1
dried pepper,1
creole season,1


In [61]:
mexican_df

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP,onion,lime,garlic,tomato,cilantro,fresh pepper,...,cumin,chili powder,chicken,salsa,corn,beef,sour cream,fish,water,canned pepper
0,Chicken Fajita Stuffed Bell Pepper,['Mexican'],"[Produce, Produce, Meat, Spices and Seasoni...","[avocado, bell pepper, chicken, chili powder, ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Corn Avocado Salsa,['Mexican'],"[Produce, Oil, Vinegar, Salad Dressing, Sp...","[avocado, vinegar, cumin, corn, garlic, bell p...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cheesy Chicken Enchilada Quinoa Casserole,['Mexican'],"[Produce, Spices and Seasonings, Canned and ...","[avocado, salt and pepper, beans, tomato, chil...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Homemade Guacamole,['Mexican'],"[Produce, Produce;Spices and Seasonings, Pro...","[avocado, cilantro, lime, salt and pepper, oni...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Mini Stuffed Mexican Bell Peppers,['Mexican'],"[Produce, Canned and Jarred, Spices and Seas...","[bell pepper, beans, chili powder, canned pepp...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Tex-Mex Polenta Rounds with Chunky Guacamole,['Mexican'],"[Produce, Spices and Seasonings, Cheese, Pr...","[avocado, chili powder, cheese, bell pepper, p...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,Fresh Mex Salsa,['Mexican'],"[Canned and Jarred, Spices and Seasonings, P...","[beans, chili powder, cilantro, cumin, corn, b...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,Salsa,['Mexican'],"[Gourmet, Produce, Pasta and Rice;Ethnic Foo...","[caviar, tomato, salsa, salsa, peach, salsa, s...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,Artichoke croutons with guacamole,['Mexican'],"[Produce, Produce, Produce;Spices and Season...","[avocado, tomato, parsley, onion, lemon, salt ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
for index, row in enumerate(mexican_ing_groups.index):
    if index < 20:
        mexican_df[row] = 0
mexican_df.columns   


Index(['recipe_name', 'cuisine_SP', 'aisle_SP', 'ingredients_SP', 'onion',
       'lime', 'garlic', 'tomato', 'cilantro', 'fresh pepper', 'cheese',
       'bell pepper', 'avocado', 'beans', 'cumin', 'chili powder', 'chicken',
       'salsa', 'corn', 'beef', 'sour cream', 'fish', 'water',
       'canned pepper'],
      dtype='object')

In [63]:
mexican_df

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP,onion,lime,garlic,tomato,cilantro,fresh pepper,...,cumin,chili powder,chicken,salsa,corn,beef,sour cream,fish,water,canned pepper
0,Chicken Fajita Stuffed Bell Pepper,['Mexican'],"[Produce, Produce, Meat, Spices and Seasoni...","[avocado, bell pepper, chicken, chili powder, ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Corn Avocado Salsa,['Mexican'],"[Produce, Oil, Vinegar, Salad Dressing, Sp...","[avocado, vinegar, cumin, corn, garlic, bell p...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cheesy Chicken Enchilada Quinoa Casserole,['Mexican'],"[Produce, Spices and Seasonings, Canned and ...","[avocado, salt and pepper, beans, tomato, chil...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Homemade Guacamole,['Mexican'],"[Produce, Produce;Spices and Seasonings, Pro...","[avocado, cilantro, lime, salt and pepper, oni...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Mini Stuffed Mexican Bell Peppers,['Mexican'],"[Produce, Canned and Jarred, Spices and Seas...","[bell pepper, beans, chili powder, canned pepp...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Tex-Mex Polenta Rounds with Chunky Guacamole,['Mexican'],"[Produce, Spices and Seasonings, Cheese, Pr...","[avocado, chili powder, cheese, bell pepper, p...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,Fresh Mex Salsa,['Mexican'],"[Canned and Jarred, Spices and Seasonings, P...","[beans, chili powder, cilantro, cumin, corn, b...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,Salsa,['Mexican'],"[Gourmet, Produce, Pasta and Rice;Ethnic Foo...","[caviar, tomato, salsa, salsa, peach, salsa, s...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,Artichoke croutons with guacamole,['Mexican'],"[Produce, Produce, Produce;Spices and Season...","[avocado, tomato, parsley, onion, lemon, salt ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
mexican_df.columns[4]

'onion'

In [65]:
# Assigning 1 or 0 if ingredient is in ingredient list
indexes = mexican_df.index.values.tolist()
for index in indexes:
    for col_name in mexican_df.columns:
        if col_name in mexican_df.ingredients_SP[index]:
            mexican_df.loc[index,col_name] = 1
mexican_df

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP,onion,lime,garlic,tomato,cilantro,fresh pepper,...,cumin,chili powder,chicken,salsa,corn,beef,sour cream,fish,water,canned pepper
0,Chicken Fajita Stuffed Bell Pepper,['Mexican'],"[Produce, Produce, Meat, Spices and Seasoni...","[avocado, bell pepper, chicken, chili powder, ...",0,0,0,0,1,0,...,1,1,1,1,0,0,0,0,0,0
1,Corn Avocado Salsa,['Mexican'],"[Produce, Oil, Vinegar, Salad Dressing, Sp...","[avocado, vinegar, cumin, corn, garlic, bell p...",0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,Cheesy Chicken Enchilada Quinoa Casserole,['Mexican'],"[Produce, Spices and Seasonings, Canned and ...","[avocado, salt and pepper, beans, tomato, chil...",1,0,0,1,1,0,...,1,1,1,0,1,0,0,0,0,0
3,Homemade Guacamole,['Mexican'],"[Produce, Produce;Spices and Seasonings, Pro...","[avocado, cilantro, lime, salt and pepper, oni...",1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Mini Stuffed Mexican Bell Peppers,['Mexican'],"[Produce, Canned and Jarred, Spices and Seas...","[bell pepper, beans, chili powder, canned pepp...",1,0,1,1,0,1,...,1,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Tex-Mex Polenta Rounds with Chunky Guacamole,['Mexican'],"[Produce, Spices and Seasonings, Cheese, Pr...","[avocado, chili powder, cheese, bell pepper, p...",1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
96,Fresh Mex Salsa,['Mexican'],"[Canned and Jarred, Spices and Seasonings, P...","[beans, chili powder, cilantro, cumin, corn, b...",1,1,0,1,1,1,...,1,1,0,0,1,0,0,0,0,0
97,Salsa,['Mexican'],"[Gourmet, Produce, Pasta and Rice;Ethnic Foo...","[caviar, tomato, salsa, salsa, peach, salsa, s...",0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
98,Artichoke croutons with guacamole,['Mexican'],"[Produce, Produce, Produce;Spices and Season...","[avocado, tomato, parsley, onion, lemon, salt ...",1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
# Dropping columns. Cleaning data for machine learning
mexican_df = mexican_df.drop(columns = ["recipe_name", "cuisine_SP", "aisle_SP",  "ingredients_SP"])
mexican_df

,onion,lime,garlic,tomato,cilantro,fresh pepper,cheese,bell pepper,avocado,beans,cumin,chili powder,chicken,salsa,corn,beef,sour cream,fish,water,canned pepper
0,0,0,0,0,1,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0
1,0,0,1,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0
2,1,0,0,1,1,0,1,0,1,1,1,1,1,0,1,0,0,0,0,0
3,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,1,0,1,1,0,1,0,1,0,1,1,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0,0,0,0,0,1,1,1,0,0,1,0,1,0,0,0,0,0,0
96,1,1,0,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0
97,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
98,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0


In [22]:
mexican_df.to_csv('100_Mexican_parsed_to_20_columns_0902.csv')